# TOC

1. [Import](#1-import)
2. [함수 정의](#2-함수-정의)
3. [Albumentation 적용 확인](#3-albumentation-적용-확인)   
    3.1. [Emboss](#31-emboss)   
    3.2. [Equalize](#32-equalize)   
    3.3. [Random contrast](#33-random-contrast)   
    3.4. [Sharpen](#34-sharpness)   
    3.5. [ToGray](#35-togray)   
    3.6. [Rotate](#36-rotate)   
    3.7. [CLAHE](#37-clahe)   
    3.8. [Horizontal Flip](#38-horizontal-flip)   
    3.9. [Center Crop](#39-centercrop)   
    3.10. [Random Scale](#310-randomscale)   
    3.11. [ShiftScaleRotate X축 이동만](#311-shiftscalerotate-x축-이동만)   
4. [생각해본 방법들](#4-생각해본-방법들)   

# 1. Import

In [ ]:
import os
import cv2
import albumentations as A
import matplotlib.pyplot as plt

# 2. 함수 정의

In [ ]:
def apply_transform_and_show(folder_name, transform, img_name):
    # train folder path
    path_train = "/opt/ml/input/data/train/DCM"
    
    path_folder = os.path.join(path_train, folder_name)

    # # 폴더 안에 이미지 파일 이름 가져오기
    # image_files = [f for f in os.listdir(path_folder) if f.endswith('.jpg') or f.endswith('.png')]

    # # 폴더 안에 랜덤하게 이미지 하나 선택(왼쪽 오른쪽 표시가 안됨)
    # img_name = random.choice(image_files)
    
    # 이미지 읽기
    img = cv2.imread(os.path.join(path_folder, img_name))
    
    # BGR to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # transform 적용s
    img_transformed = transform(image=img)["image"]
    
    # original image & transformed image
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    
    ax[0].imshow(img)
    ax[0].set_title(f"Original Image {folder_name}")
    
    ax[1].imshow(img_transformed)
    ax[1].set_title(f"Transformed Image {folder_name}")
    
    plt.show()

# 3. Albumentation 적용 확인

## 3.1 Emboss

In [ ]:
# 적용할 transform 정의
transform = A.Compose([
    A.Resize(512,512),
    A.Emboss(alpha=(0.5, 0.6), strength=(1.3, 1.3), p=1.0),
])

In [ ]:
apply_transform_and_show("ID363", transform, "image1664935962797.png")

## 3.2 Equalize

In [ ]:
transform = A.Compose([
    A.Equalize() # Equalize the image histogram.
    # mask parameter에 대해서 더 알아보기
])
apply_transform_and_show("ID001", transform)

## 3.3 Random Contrast

In [ ]:
transform = A.Compose([
    A.RandomContrast(limit=[0.7,0.7], p=1.0) # -0.3~0.3 사이로 랜덤한 contrast 
])
apply_transform_and_show("ID001", transform)

## 3.4 Sharpness

In [ ]:
transform = A.Compose([
    A.Sharpen(alpha=(0.5, 0.9), lightness=(2.0, 2.0), p=1.0)
])
apply_transform_and_show("ID001", transform)

## 3.5 ToGray

In [ ]:
transform = A.Compose([
    A.ToGray(p=1.0)
])
apply_transform_and_show("ID001", transform)

## 3.6 Rotate

<br>

* 테스트나 train 이미지 확인해서 왼손과 오른손에 따라 어느 방향으로 돌리는지 확인 할 필요 있을 듯! 

<br>

* train에서 이미 돌려진 손에 rotate를 적용할 필요가 있을까? -> rotate를 offline으로 적용(rotate 안된 이미지만 골라서)

In [ ]:
transform = A.Compose([
    A.Rotate (limit=[-35,-35], interpolation=1, border_mode=1, p=1.0)
    
])
apply_transform_and_show("ID001", transform)

## 3.7 CLAHE

In [ ]:
transform = A.Compose([
    A.CLAHE(clip_limit=[3.0,3.0], tile_grid_size=(8, 8), p=1.0)
])
apply_transform_and_show("ID001", transform)

## 3.8 Horizontal Flip

In [ ]:
transform = A.Compose([
    A.HorizontalFlip(p=1.0)
])
apply_transform_and_show("ID001", transform)

## 3.9 CenterCrop
<br>

* 손이 잘리는 경우 조심

<br>

* rotate된 손에 center crop 할 경우 정보 손실 우려

<br>

* TTA에 centercrop 어려울듯 (rotate 된 손)

In [ ]:
transform = A.Compose([
    A.CenterCrop(2000, 1400, always_apply=True, p=1.0)
])
apply_transform_and_show("ID001", transform)

## 3.10 RandomScale

In [ ]:
transform = A.Compose([
    A.RandomScale (scale_limit=[0.3,0.3], interpolation=1, always_apply=True, p=1.0)
])
apply_transform_and_show("ID001", transform)

## 3.11 ShiftScaleRotate (x축 이동만)

In [ ]:
transform = A.Compose([
    A.ShiftScaleRotate (shift_limit=[0.1,0.15], scale_limit=0.0, rotate_limit=0,
                        interpolation=1, border_mode=1, value=None, mask_value=None,
                        shift_limit_x=None, shift_limit_y=0.0,
                        always_apply=False, p=1.0)
    # 0.1 ~ 0.15 사이의 범위로 x축 shift
])
apply_transform_and_show("ID001", transform)

# 4. 생각해본 방법들

1. Transfer Learning: Offline cutmix -> Offline mosaic(Object Detection에서 큰 성능 향상)

<br>

2. Transfer Learning: Heavy Augmentation -> Low Augmentation(필요한 augmentation만)

<br>

3. 잘 맞추지 못하는 영역만 Crop 해서 이미지로 만들고 -> dataset에 추가(일종의 Offline Augmentation)

<br>

4. 찾아본 방법들
- RICAP(random image cropping and patching) -> mosaic과 비슷한듯?
- mixup
- https://github.com/yizhezhang2000/SAMAug (Segment Anything을 이용한 의료 데이터 증폭?)
-> https://arxiv.org/abs/2304.11332